In [2]:
# Importando bibliotecas necessárias
import cv2
import pandas as pd
from datetime import datetime
import os
import numpy as np

In [ ]:
# Função para obter máscara de movimento limpa
def get_motion_mask(fg_mask, kernel=np.ones((9, 9), np.uint8)):
    # Aplica threshold para obter uma imagem binária
    _, thresh = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)
    # Aplica mediana para reduzir o ruído
    motion_mask = cv2.medianBlur(thresh, 5)
    # Aplica operações morfológicas de abertura e fechamento
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel)
    return motion_mask

# Função para realizar supressão de não-máximos
def non_max_suppression(bboxes, scores, threshold=0.3):
    x1 = bboxes[:, 0]
    y1 = bboxes[:, 1]
    x2 = bboxes[:, 2]
    y2 = bboxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= threshold)[0]
        order = order[inds + 1]

    return keep

# Função principal para processar o vídeo e detectar movimento
def process_video(video_path, skip_frames=5, output_dir='/content/drive/My Drive/animal_detection_output'):
    motion_list = [None, None]
    contour_area_threshold = 10000
    var_threshold = 50
    use_hist_eq = False

    # Inicializa o subtrator de fundo MOG2 com parâmetros ajustados
    back_sub = cv2.createBackgroundSubtractorMOG2(varThreshold=var_threshold, detectShadows=True)

    # Abre o vídeo
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Erro ao abrir o vídeo {video_path}")
        return

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_output_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    frame_count = 0
    frames = []

    while True:
        check, frame = video.read()
        if not check:
            break

        frame_count += 1
        if frame_count % skip_frames != 0:
            continue

        # Converte o frame para escala de cinza
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if use_hist_eq:
            gray = cv2.equalizeHist(gray)

        # Aplica a subtração de fundo MOG2
        fg_mask = back_sub.apply(gray)
        frames.append(frame)  # Armazena os frames coloridos
        motion_mask = get_motion_mask(fg_mask)

        # Encontra contornos na máscara de movimento
        cnts, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        for contour in cnts:
            if cv2.contourArea(contour) < contour_area_threshold:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            detections.append((x, y, x+w, y+h))

        # Aplica supressão de não-máximos
        detections = np.array(detections)
        if len(detections) > 0:
            bboxes = detections[:, :4]
            scores = np.ones((len(bboxes),))  # placeholder for scores
            keep = non_max_suppression(bboxes, scores, threshold=0.3)
            for i in keep:
                (x, y, x2, y2) = bboxes[i]
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 3)

        motion_list.append(len(detections) > 0)
        motion_list = motion_list[-2:]

        if len(detections) > 0:
            frame_path = os.path.join(video_output_dir, f'frame_{frame_count}.jpg')
            cv2.imwrite(frame_path, frame)
            print(f"Frame {frame_count} salvo com movimento detectado.")

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    if frames:
        # Calcula o background como a mediana dos frames coloridos
        background = np.median(frames, axis=0).astype(np.uint8)
        background_path = os.path.join(video_output_dir, 'background.jpg')
        cv2.imwrite(background_path, background)
        print(f"Background salvo em {background_path}")

    video.release()
    cv2.destroyAllWindows()
    print(f"Processamento do vídeo {video_path} concluído.")

In [ ]:
# Listar todos os arquivos de vídeo na pasta especificada
video_extensions = ('.mov', '.mp4', '.avi', '.mkv')
video_paths = [os.path.join(videos_dir, f) for f in os.listdir(videos_dir) if f.lower().endswith(video_extensions)]

In [ ]:
# Processa cada vídeo
for video_path in video_paths:
    print(f"Iniciando processamento do vídeo {video_path}")
    process_video(video_path, skip_frames=10, output_dir=output_dir)
    print(f"Processamento do vídeo {video_path} finalizado.")